In [ ]:
params = {
    'num_clusters': 50,
    'num_components': 256,
    'image_size': (384, 380),
    'random_state': 123,

    'lr': 1e-4,
    'max_lr': 1e-4,
    'epochs': 100,
    'batch_size': 16,
    'print_interval': 0.3,
    
    'save_path': 'CNN-k50.pt',
    'use_pretrained': True,

    'wandb_key': '6f927fe3835ebcc7bb05946984340ac2c810388e',
    'wandb_run_name': 'Run 1 (k = 50, b4, pretrained)'
}

In [ ]:
!pip install -q --upgrade torchvision
!pip install -q --upgrade datasets

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import yaml
import torch
import torchvision
from torch import optim, nn
from torchvision import models, transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import cv2
from PIL import Image
import os
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import seaborn as sns
import plotly.express as px
from datasets import Dataset
import datasets
from sklearn.decomposition import PCA
import wandb
import os
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import average_precision_score
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader,Dataset
from sklearn.metrics import f1_score
# datasets.disable_progress_bar()

In [ ]:
def read_config(path = 'config.yml'):
    with open('../input/thesis-chatbot/' + path) as f:
        config =  yaml.safe_load(f)
    return config

def get_train_imgs(img_ls):
    """
    Lấy ảnh ở tập train ra, kiểm tra xác thực đảm bảo 
    nó có trong folder chứa ảnh
        
        Params:
            img_ls (pandas series): danh sách ảnh train/val

        Returns: 
            temp (list): danh sách ảnh output
    """
    img_train = img_ls.dropna().values
    img_train = list(set(img_train))

    img_folder = os.listdir('../input/thesis-chatbot/' + config['path']['image_path'])

    temp = []
    for img in img_train:
        if img in img_folder:
            temp.append(img)
    
    return temp


def feed_img(model, img_file, image_size):
    """
    Đưa 1 ảnh qua mô hình
        Params: 
            model (pytorch module): mô hình
            img_file (str): tên file ảnh
            image_size (int, int): kích thước ảnh sau khi resize cho input mô hình
            
    """
    device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    filename = '../input/thesis-chatbot/' + str(config['path']['image_path']) + str(img_file)
    # try:
    img_file = Image.open(str(filename))
    
    # Gif 
    img_file = np.array(img_file.convert('RGB'))
  
    # Return None nếu file ảnh không đọc được
    if img_file is None:
        #Dùng None thay vì continue, continue thì xuống dưới hong bỏ được tên ảnh mà feature nó bị None
        #check if feature not None, mới cho lấy tên ảnh, feature --> làm luôn 2 việc.
        return None  

    # Transform the image
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.CenterCrop(512),
        transforms.Resize(image_size),
        transforms.ToTensor()                              
    ])

    img_file = transform(img_file)

    # Reshape the image. PyTorch model reads 4-dimensional tensor
    # [batch_size, channels, width, height]
    img_file = img_file.reshape(1, 3, image_size[0], image_size[1])
    img_file = img_file.to(device)
    # We only extract features, so we don't need gradient
    
    with torch.no_grad():
        model.eval()
    # Extract the feature from the image
        feature = model(img_file)
    # Convert to NumPy Array, Reshape it, and save it to features variable
    return feature.cpu().detach().numpy().reshape(-1)

def feed_single_img(model, example):
    """
    Mỗi step sẽ làm gì - feed ảnh qua mô hình
    Feature được lưu ở cột features
        Params:
            model (Pytorch module): mô hình CNN
            example: 1 dòng trong dataframe
        Returns:
            example: 1 dòng trong dataframe (đã thêm cột features)
    """
    example['features'] = feed_img(model, example['img'], params['image_size'])
    return example

def get_img_features_faster(img_train):
    """
    Đưa toàn bộ ảnh qua mô hình - xử lí song song
    Tên ảnh được lưu ở cột 'img'
    Feature được lưu ở cột 'features'
        Params:
            model (pytorch module): mô hình CNN ở biến global
            img_train (list): danh sách tên ảnh train
        Returns:
            temp (huggingface dataset): class dataset của huggingface
                    Giống dictionary
    """
    global model
    temp = pd.DataFrame({'img': img_train})
    temp = datasets.Dataset.from_pandas(temp)
    temp = temp.map(lambda x: feed_single_img(model, x))
    return temp

def kmean_fit(features, n_clusters, random_state = 42):
    """
    Run kmean clustering
        Params:
            features (list): list các features ảnh trong tập train
            n_clusters: số cluster
            random_state: fixed random state
        
        Returns:
            labels (list): list nhãn giả tương ứng với features
    """
    # Initialize the model
    kmean_model = KMeans(n_clusters=n_clusters, random_state=random_state)
    # Fit the data into the model
    kmean_model.fit(features)
    # Extract the labels
    labels = kmean_model.labels_
    return labels

def make_pseudo_labels(img_train, num_components, num_clusters):
    # Feed lấy image features
    img_dataset = get_img_features_faster(img_train)

    # PCA rút gọn features
    pca = PCA(n_components= num_components, random_state= params['random_state'])
    pca_data = pca.fit_transform(img_dataset['features'])

    # Kmean clustering
    labels = kmean_fit(pca_data, n_clusters = num_clusters, random_state = params['random_state'])
    return labels, img_dataset

class MyData(Dataset):

    def __init__(self, img_dataset, labels, transform):
        self.img_dataset = img_dataset
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(img_dataset)

    def __getitem__(self, index):
        image = Image.open('../input/thesis-chatbot/' + config['path']['image_path'] + img_dataset['img'][index])
        image = np.array(image.convert('RGB'))

        if self.transform is not None:
            image = self.transform(image)
        labels = self.labels[index]

        return {'images': image, 'labels': labels}

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
def remove_classifier(model):
    """
    Tháo lớp classifier ra khỏi mô hình 
        Params:
            model (pytorch_module): mô hình
                pass by reference, thay đổi sẽ ảnh hưởng thẳng tới biến mô hình
                !!! classifier sẽ là attribute model.classifier
                --> mình đè lên bằng class Identity - class fake lấy output là ouput của lớp trước nó
        Returns:
            None
    """

    # Class Identity: class fake không có gì ở trong, 
    # lấy output của lớp trước đó output ra luôn


    model.classifier[1] = Identity()

def add_classifier(model, last_layer_shape: int, num_classes: int):
    """
    Gắn lớp classifier vào mô hình
        Params:
            model (pytorch module): mô hình
                pass by reference, thay đổi sẽ ảnh hưởng thẳng tới biến
                !!! module mô hình có attribute model.classifier, 
               
            last_layer_shape (int): kích thước đầu ra của layer trước đó 
                    (image representation layer)

            num_classes (int): số class đầu ra cho classification
    """

    model.classifier[1] = nn.Linear(last_layer_shape, num_classes)
    

# METRIC----------------------
def accuracy(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return torch.mean((classes == labels).float())

def nmi(predictions, labels, is_prob = True):
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html
    if is_prob:
        classes = torch.argmax(predictions, dim=1)
    else:
        classes = predictions
    return normalized_mutual_info_score(classes, labels)

def f1(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return f1_score(labels, classes, average='weighted')

In [ ]:
# Load config
config = read_config()
# Load df
df = pd.read_pickle('../input/thesis-chatbot/' + config['path']['train_preprocessed_path'])
# Get list of train imgs
img_train = get_train_imgs(df['img_id'])
# Load model
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

# Update model
if params['use_pretrained']:
    model = models.efficientnet_b4(pretrained=True)
else:
    model = models.efficientnet_b4()
# Kích thước của lớp biểu diễn đặc trưng ảnh
# b4: 1408, b0: 1280 tuỳ version của model. Mình set bằng biến cho linh hoạt
projection_shape = model.classifier[1].in_features  
model.classifier = nn.Sequential(
                                nn.Linear(projection_shape, 768),
                                nn.Linear(768, params['num_clusters'])
                                )

#Transform tăng cường ảnh 
transform_aug = transforms.Compose([
                transforms.ToPILImage(),
                transforms.CenterCrop(512),
                transforms.Resize(params['image_size']),
                transforms.RandomChoice([ # random 1 phép để dùng
                    transforms.RandomHorizontalFlip(p=0.5), #lật ngang
                    transforms.RandomRotation(degrees=30),  #xoay góc 30
                    # thay đổi độ sáng, tương phản, bão hoà, hue
                    transforms.ColorJitter(brightness= (0.1, 1), contrast= (0.1, 1), saturation= (0.1, 1), hue= (-0.1, 0.1)),
                    transforms.RandomPerspective(), #quay góc nhìn
                ]),
                transforms.ToTensor()])

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr= params['lr'])
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
                                                # max_lr= params['max_lr'], 
                                                # steps_per_epoch=len(train_loader), 
                                                # epochs= params['epochs'])

os.environ["WANDB_API_KEY"] = params['wandb_key']
wandb.init(config = params, project="CNN brrr", entity="thesis-chatbot", name = params['wandb_run_name'])

In [ ]:
for epoch in tqdm(range(1, params['epochs'] + 1)):
    print(f"\n\nEpoch {epoch}")
    print("=" * 10)

    # 1. Remove classifier
    img_train = get_train_imgs(df['img_id'])
    remove_classifier(model)

    # 2. Clustering
    print("Clustering...")
    model.eval() #bật chế độ inference (tắt dropout)
    
    # Nếu epoch > 1, Tính NMI giữa pseudo labels của epoch hiện tại vs pseudo labels của epoch trước đó
    if epoch > 1:
        previous_labels = labels
        labels, img_dataset = make_pseudo_labels(img_train, 
               num_components = params['num_components'],
               num_clusters = params['num_clusters'])
        reassign_nmi = nmi(labels, previous_labels, is_prob = False)
        print(f"Reassigned NMI: {reassign_nmi}")
        wandb.log({'Reassigned NMI': reassign_nmi})
        
    else:
        labels, img_dataset = make_pseudo_labels(img_train, 
                   num_components = params['num_components'],
                   num_clusters = params['num_clusters'])


    # 3. Dataset + Dataloader
    train_set = MyData(img_dataset, labels, transform_aug)
    train_loader = torch.utils.data.DataLoader(train_set, 
                            batch_size=params['batch_size'], 
                            shuffle=True)

    # 4. Add classifier
    add_classifier(model, 768, params['num_clusters'])
    model = model.to(device) # đưa dô gpu (nếu có)

    # 5. Classification
    print("Training...")
    running_loss = []
    running_accuracy = []
    running_nmi = []
    running_f1 = []
    

    for i, data in enumerate(tqdm(train_loader)):
        model.train()
        inputs, labels_int = data['images'].to(device), data['labels'].to(device)  

        # reset gradients đã tính mỗi batch
        optimizer.zero_grad()
        
        # forward 
        outputs = model(inputs)
        long_labels = labels_int.type(torch.LongTensor).to(device)

        # Loss
        loss = criterion(outputs, long_labels)

        # backward
        loss.backward()

        # Gradient clipping - prevent exploding gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm= 1.0)

        # Update weights
        optimizer.step()

        # Đưa loss, accuracy
        running_loss.append(loss.item())
        running_accuracy.append(accuracy(outputs, long_labels))

        # Hàm nmi cần nhận vào các biến ở cpu
        running_nmi.append(nmi(outputs.detach().cpu(), long_labels.detach().cpu(), is_prob = True))
        running_f1.append(f1(outputs.detach().cpu(), long_labels.detach().cpu()))
    
    
    # Do train 1 epoch quá lẹ nên chỉ cần lấy metric cuối epoch
    # Loss
    last_loss = torch.mean(torch.Tensor(running_loss))
    # Accuracy
    last_accuracy = torch.mean(torch.Tensor(running_accuracy))
    # Pred NMI
    last_nmi = torch.mean(torch.Tensor(running_nmi))
    # f1
    last_f1 = torch.mean(torch.Tensor(running_f1))

    # In ra
    print(f"Epoch {epoch} ({i + 1}/{len(train_loader)}), Loss: {last_loss:.3f}, Accuracy: {last_accuracy:.3f}, Pred NMI: {last_nmi:.3f}, F1: {last_f1:.3f}")

    # Log qua wandb, 4 plots
    wandb.log({"train/loss": last_loss, 
                'train/accuracy': last_accuracy,
                'train/Pred NMI': last_nmi,
                'train/f1': last_f1})
    
    # Luu
    torch.save(model.state_dict(), params['save_path'])